# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [48]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, countDistinct, sum
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
sparkify_data = 'mini_sparkify_event_data.json'
df = spark.read.json(sparkify_data)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# Total sessions
df.count()

286500

In [17]:
# Count User
df.agg(countDistinct('userId')).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   226|
+----------------------+



In [62]:
df.where(df.userId == '').count()

8346

In [50]:
df.groupby('auth').agg(F.countDistinct('userId')).show()

+----------+----------------------+
|      auth|count(DISTINCT userId)|
+----------+----------------------+
|Logged Out|                     1|
| Cancelled|                    52|
|     Guest|                     1|
| Logged In|                   225|
+----------+----------------------+



In [51]:
df.filter(df.auth == 'Cancelled').take(5)

[Row(artist=None, auth='Cancelled', firstName='Adriel', gender='M', itemInSession=104, lastName='Mendoza', length=None, level='paid', location='Kansas City, MO-KS', method='GET', page='Cancellation Confirmation', registration=1535623466000, sessionId=514, song=None, status=200, ts=1538943990000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"', userId='18'),
 Row(artist=None, auth='Cancelled', firstName='Diego', gender='M', itemInSession=56, lastName='Mckee', length=None, level='paid', location='Phoenix-Mesa-Scottsdale, AZ', method='GET', page='Cancellation Confirmation', registration=1537167593000, sessionId=540, song=None, status=200, ts=1539033046000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='32'),
 Row(artist=None, auth='Cancelled', firstName='Mason', gender='M', itemInSession=10, la

In [65]:
#Downgrade, Submit Downgrade
pages = df.groupby('page','method').count().sort('method')
pages_df = pages.toPandas()

In [68]:
# import plotly.graph_objects as go
import plotly.express as px
pages_df_plt = pages_df.loc[pages_df['method'] == 'PUT']

ModuleNotFoundError: No module named 'plotly.express'

In [8]:
df.groupby('status').count().show()

+------+------+
|status| count|
+------+------+
|   307| 26430|
|   404|   258|
|   200|259812|
+------+------+



In [44]:
df.select('userAgent').head()

Row(userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0')

In [10]:
df.filter(df.userId == '30').select(['artist','auth','method','location','registration','page']).show()

+--------------------+---------+------+---------------+-------------+---------------+
|              artist|     auth|method|       location| registration|           page|
+--------------------+---------+------+---------------+-------------+---------------+
|      Martha Tilston|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|        Adam Lambert|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|           Daft Punk|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|        Starflyer 59|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|                null|Logged In|   PUT|Bakersfield, CA|1538173362000|Add to Playlist|
|            Frumpies|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|Edward Sharpe & T...|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|         Stan Mosley|Logged In|   PUT|Bakersfield, CA|1538173362000|       NextSong|
|             Orishas|Logged In|   PUT|Bakersfield, CA

In [45]:
# Downgrade Sample
df.filter(col('userId') == '131').groupby(['level','sessionId','page']\
                                         ).agg(min(col('ts')).alias('ts'),\
                                              sum(col('length')).alias('length')).sort(col('ts').desc()).show(50)

+-----+---------+----------------+---------+-------------+------------------+
|level|sessionId|            page|     auth|           ts|            length|
+-----+---------+----------------+---------+-------------+------------------+
| free|     2281|            Home|Logged In|1543548083000|              null|
| free|     2155|      Add Friend|Logged In|1543120417000|              null|
| free|     2155|     Thumbs Down|Logged In|1543117205000|              null|
| free|     2155|            Home|Logged In|1543114842000|              null|
| free|     2155|          Logout|Logged In|1543114535000|              null|
| free|     2155|     Roll Advert|Logged In|1543114106000|              null|
| free|     2155|       Thumbs Up|Logged In|1543111791000|              null|
| free|     2155|           About|Logged In|1543110822000|              null|
| free|     2155| Add to Playlist|Logged In|1543109614000|              null|
| free|     2155|        NextSong|Logged In|1543109551000|11182.

In [ ]:
# define churn
# Downgrade: Submit Downgrade
# Cancel: 

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.